In [42]:
import pandas as pd
import numpy as np
from arch import arch_model

In [43]:
df = pd.read_csv(r'C:\Users\leoqi\Desktop\NUS_Modules\DSE4211\DSE4211-Group-BTC-Day-Trading\data\processed\combined_df.csv')

In [44]:
df

,Unnamed: 0,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,...,%D,Stochastic_Signal,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR
0,0,2024-03-02 02:00:00+00:00,0.157746,-188.470966,0.000000,62030.01,62216.71,61949.03,62111.18,56088,...,55.291400,0,1153.46042,-263.362181,1,267.68,186.70,80.98,267.68,384.756667
1,1,2024-03-02 03:00:00+00:00,0.092267,-204.409405,0.552411,62111.18,62426.70,62025.00,62307.45,67185,...,55.447096,0,1465.47878,-98.710661,0,401.70,315.52,86.18,401.70,347.451667
2,2,2024-03-02 04:00:00+00:00,0.050600,-205.344729,0.000000,62307.45,62307.45,62035.95,62035.95,56098,...,55.571070,0,-1147.71004,-198.615364,-1,271.50,0.00,271.50,271.50,333.268333
3,3,2024-03-02 05:00:00+00:00,0.026060,-200.706776,0.000000,62035.95,62079.99,61752.30,62006.55,65329,...,51.354794,0,-1415.65823,-314.524208,0,327.69,44.04,283.65,327.69,326.113333
4,4,2024-03-02 06:00:00+00:00,0.012619,-185.091972,1.010990,62006.55,62256.00,61930.00,61946.00,57826,...,30.839202,0,-1059.53378,-385.477501,0,326.00,249.45,76.55,326.00,325.211667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,...,74.485905,0,208.35672,63.377330,1,150.00,109.25,40.75,150.00,298.715000
7926,7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,...,70.787554,0,283.50224,84.341607,0,358.05,223.01,135.04,358.05,325.368333
7927,7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,...,49.851126,0,-621.23723,17.143622,-1,586.40,0.01,586.39,586.40,326.935000
7928,7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,...,33.422136,0,252.60356,39.568378,1,222.90,92.87,130.03,222.90,277.221667


In [45]:
df['log_return'] = np.log(df['CLOSE'] / df['CLOSE'].shift(1))
df.dropna(inplace=True)

In [46]:
df['log_return']

1       0.003155
2      -0.004367
3      -0.000474
4      -0.000977
5       0.001062
          ...   
7925    0.000072
7926    0.000305
7927   -0.003819
7928    0.000815
7929    0.000987
Name: log_return, Length: 7929, dtype: float64

In [47]:
df['log_return_100'] = df['log_return'] * 100

In [48]:
for start in range(500, len(df)):
    # Slice the rolling window data (500 previous returns)
    rolling_data = df['log_return_100'].iloc[start - 500:start]
    
    # Fit the GARCH model
    model = arch_model(rolling_data, vol='GARCH', p=1, q=1)
    model_fitted = model.fit(disp='off')
    
    # Forecast volatility for the next period
    forecast = model_fitted.forecast(horizon=1)
    predicted_volatility = np.sqrt(forecast.variance.values[-1, 0])
    
    # Store the predicted volatility in the corresponding row of the DataFrame
    df.loc[df.index[start], 'predicted_conditional_volatility'] = predicted_volatility/100

In [49]:
df

,Unnamed: 0,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,...,OBV,OBV_EMA20,OBV_Signal,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility
1,1,2024-03-02 03:00:00+00:00,0.092267,-204.409405,0.552411,62111.18,62426.70,62025.00,62307.45,67185,...,1465.47878,-98.710661,0,401.70,315.52,86.18,401.70,347.451667,0.315500,NaN
2,2,2024-03-02 04:00:00+00:00,0.050600,-205.344729,0.000000,62307.45,62307.45,62035.95,62035.95,56098,...,-1147.71004,-198.615364,-1,271.50,0.00,271.50,271.50,333.268333,-0.436695,NaN
3,3,2024-03-02 05:00:00+00:00,0.026060,-200.706776,0.000000,62035.95,62079.99,61752.30,62006.55,65329,...,-1415.65823,-314.524208,0,327.69,44.04,283.65,327.69,326.113333,-0.047403,NaN
4,4,2024-03-02 06:00:00+00:00,0.012619,-185.091972,1.010990,62006.55,62256.00,61930.00,61946.00,57826,...,-1059.53378,-385.477501,0,326.00,249.45,76.55,326.00,325.211667,-0.097699,NaN
5,5,2024-03-02 07:00:00+00:00,0.005751,-164.865815,0.000000,61946.00,62041.70,61790.98,62011.85,86425,...,1223.45307,-232.246018,1,250.72,95.70,155.02,250.72,307.548333,0.106246,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,...,208.35672,63.377330,1,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691
7926,7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,...,283.50224,84.341607,0,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626
7927,7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,...,-621.23723,17.143622,-1,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572
7928,7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,...,252.60356,39.568378,1,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957


In [50]:
df = df.dropna()

In [51]:
df['rolling_mean_volatility'] = df['predicted_conditional_volatility'].rolling(window=20).mean()
df['rolling_std_volatility'] = df['predicted_conditional_volatility'].rolling(window=20).std()

df['volatility_category'] = np.where(df['predicted_conditional_volatility'] > df['rolling_mean_volatility'] + df['rolling_std_volatility'], 'High',
                                    np.where(df['predicted_conditional_volatility'] < df['rolling_mean_volatility'] - df['rolling_std_volatility'], 'Low', 'Normal'))

C:\Users\leoqi\AppData\Local\Temp\ipykernel_25036\3504656199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rolling_mean_volatility'] = df['predicted_conditional_volatility'].rolling(window=20).mean()
C:\Users\leoqi\AppData\Local\Temp\ipykernel_25036\3504656199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rolling_std_volatility'] = df['predicted_conditional_volatility'].rolling(window=20).std()
C:\Users\leoqi\AppData\Local\Temp\ipykernel_25036\3504656199.py:4: SettingWithCopyWarning: 
A 

In [52]:
df

,Unnamed: 0,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,...,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category
501,501,2024-03-22 23:00:00+00:00,-3.216234,423.485031,0.000000,63022.00,63812.00,62985.00,63796.64,76452,...,827.00,790.00,37.00,827.00,768.191667,1.221665,0.007813,NaN,NaN,Normal
502,502,2024-03-23 00:00:00+00:00,-3.087048,586.199779,0.000000,63796.64,63804.00,63536.30,63571.99,67468,...,267.70,7.36,260.34,267.70,723.788333,-0.352756,0.008774,NaN,NaN,Normal
503,503,2024-03-23 01:00:00+00:00,-3.020001,704.647552,0.000000,63571.99,64334.25,63544.00,64111.63,91438,...,790.25,762.26,27.99,790.25,741.196667,0.845282,0.008206,NaN,NaN,Normal
504,504,2024-03-23 02:00:00+00:00,-2.819386,747.533415,0.000000,64111.63,64374.91,63739.00,63861.79,74982,...,635.91,263.28,372.63,635.91,743.346667,-0.390457,0.008283,NaN,NaN,Normal
505,505,2024-03-23 03:00:00+00:00,-2.515302,709.865145,1.044452,63861.79,63866.00,63000.00,63556.00,86348,...,866.00,4.21,861.79,866.00,678.155000,-0.479981,0.007786,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,...,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal
7926,7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,...,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low
7927,7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,...,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low
7928,7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,...,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High


In [53]:
df = df.dropna()

In [54]:
df

,Unnamed: 0,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,...,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category
520,520,2024-03-23 18:00:00+00:00,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,...,411.71,29.02,382.69,411.71,658.445000,-0.425594,0.005914,0.006518,0.001199,Normal
521,521,2024-03-23 19:00:00+00:00,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,...,698.90,111.27,587.63,698.90,701.053333,-0.769282,0.005821,0.006418,0.001168,Normal
522,522,2024-03-23 20:00:00+00:00,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,...,445.27,245.30,199.97,445.27,604.058333,0.210775,0.006339,0.006296,0.001028,Normal
523,523,2024-03-23 21:00:00+00:00,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,...,419.97,190.19,229.78,419.97,571.245000,-0.100174,0.005951,0.006184,0.000926,Normal
524,524,2024-03-23 22:00:00+00:00,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,...,306.17,106.03,200.14,306.17,468.430000,-0.303533,0.005584,0.006049,0.000790,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,7925,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,...,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal
7926,7926,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,...,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low
7927,7927,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,...,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low
7928,7928,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,...,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High


In [55]:
df.to_csv(r'C:\Users\leoqi\Desktop\NUS_Modules\DSE4211\DSE4211-Group-BTC-Day-Trading\data\processed\combined_df_with_volatility.csv', index=False)